# Notebook

In [1]:
# installs- comment out if good

!git clone https://github.com/wtlu71/cnn-comp-med.git

import numpy as np
import os
from tqdm import tqdm
import sys
import os

repo_path = '/content/cnn-comp-med'
# Add to Python path
sys.path.append(repo_path)
# imports
from my_scripts.test import potato
from my_scripts.my_models import SmallCNN, SmallMLP
from my_scripts.dataset_loading import H5Dataset
from my_scripts.utils import run_epoch

import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader, Subset
import torchvision.models as models
from torchvision.models import resnet50
from torchvision import datasets, transforms

device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'
# Path to your repo in Drive

Cloning into 'cnn-comp-med'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (86/86), done.
remote: Compressing objects: 100% (72/72), done.
Receiving objects: 100% (86/86), 41.07 KiB | 5.87 MiB/s, done.
remote: Total 86 (delta 42), reused 33 (delta 10), pack-reused 0 (from 0)
Resolving deltas: 100% (42/42), done.


In [2]:
print(potato(2, 4))

6


Get train/val/test data from Zenodo:

In [3]:
os.makedirs(os.path.join(os.getcwd(),"data"),exist_ok=True)

In [4]:
# !apt-get install -y wget2
!apt-get install -y aria2

# !aria2c -x 16 -s 16 -k 1M -c -j 1 "URL" -o output_filename
#train
!aria2c -x 16 -s 16 -k 1M -c -j 1 "https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_x.h5.gz?download=1" -o data/train_x.h5.gz
!gunzip data/train_x.h5.gz

!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_y.h5.gz?download=1 -O data/train_y.h5.gz
!gunzip data/train_y.h5.gz

# val
!aria2c -x 16 -s 16 -k 1M -c -j 1 "https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz?download=1" -o data/valid_x.h5.gz
!gunzip data/valid_x.h5.gz

!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_valid_y.h5.gz?download=1 -O data/valid_y.h5.gz
!gunzip data/valid_y.h5.gz

# test
!aria2c -x 16 -s 16 -k 1M -c -j 1 "https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_test_x.h5.gz?download=1" -o data/test_x.h5.gz
!gunzip data/test_x.h5.gz

!wget https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_test_y.h5.gz?download=1 -O data/test_y.h5.gz
!gunzip data/test_y.h5.gz

# # train
# !wget2 --progress=bar:force https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_x.h5.gz?download=1 -O data/train_x.h5.gz
# !gunzip data/train_x.h5.gz

# !wget2 --progress=bar:force https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_train_y.h5.gz?download=1 -O data/train_y.h5.gz
# !gunzip data/train_y.h5.gz

# # val
# !wget2 --progress=bar:force https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_valid_x.h5.gz?download=1 -O data/valid_x.h5.gz
# !gunzip data/valid_x.h5.gz

# !wget2 --progress=bar:force https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_valid_y.h5.gz?download=1 -O data/valid_y.h5.gz
# !gunzip data/valid_y.h5.gz

# # test
# !wget2 --progress=bar:force https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_test_x.h5.gz?download=1 -O data/test_x.h5.gz
# !gunzip data/test_x.h5.gz

# !wget2 --progress=bar:force https://zenodo.org/records/2546921/files/camelyonpatch_level_2_split_test_y.h5.gz?download=1 -O data/test_y.h5.gz
# !gunzip data/test_y.h5.gz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libaria2-0 libc-ares2
The following NEW packages will be installed:
  aria2 libaria2-0 libc-ares2
0 upgraded, 3 newly installed, 0 to remove and 41 not upgraded.
Need to get 1,513 kB of archives.
After this operation, 5,441 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libc-ares2 amd64 1.18.1-1ubuntu0.22.04.3 [45.1 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libaria2-0 amd64 1.36.0-1 [1,086 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 aria2 amd64 1.36.0-1 [381 kB]
Fetched 1,513 kB in 2s (680 kB/s)
Selecting previously unselected package libc-ares2:amd64.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../libc-ares2_1.18.1-1ubuntu0.22.04.3_amd64.deb ...
Unpacking libc-ares2:amd64 (1.18.1-1ubuntu

In [9]:
# define transforms

# is there a way to get this programmatically from the data or not worth it
IMG_SIZE = 96
BATCH_SIZE = 2048

# Training transforms with augmentation
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(10),
    # TODO: normalize: mean and std for ImageNet (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Validation/Test transforms (no augmentation)
eval_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    # TODO: normalize: mean and std for ImageNet (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Data transforms defined")

Data transforms defined


In [10]:
# dataset paths- Colab virtual session
train_img_h5_path = "data/train_x.h5"
train_label_h5_path = "data/train_y.h5"

val_img_h5_path = "data/valid_x.h5"
val_label_h5_path = "data/valid_y.h5"

test_img_h5_path = "data/test_x.h5"
test_label_h5_path = "data/test_y.h5"

train_subset_size = 50
eval_subset_size = 10

train_dataset = H5Dataset(train_img_h5_path,train_label_h5_path,transform=train_transforms)
# train_dataset_subset = Subset(train_dataset, range(train_subset_size))
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

val_dataset = H5Dataset(val_img_h5_path,val_label_h5_path,transform=train_transforms)
# val_dataset_subset = Subset(val_dataset, range(eval_subset_size))
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

test_dataset = H5Dataset(test_img_h5_path,test_label_h5_path,transform=train_transforms)
# test_dataset_subset = Subset(test_dataset, range(eval_subset_size))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

if torch.cuda.is_available():
    print("Using GPU")
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
else:
    print("Using CPU")
    # train_dataset_subset = Subset(train_dataset, range(train_subset_size))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    # val_dataset_subset = Subset(val_dataset, range(eval_subset_size))
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    # test_dataset_subset = Subset(test_dataset, range(eval_subset_size))
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

Using GPU


In [15]:
# define/load models
mlpmodel = SmallMLP().to(device)
smallcnnmodel = SmallCNN().to(device)
# resnetmodel = resnet50(weights = models.ResNet50_Weights.IMAGENET1K_V2).to(device)
resnetmodel = resnet50().to(device)

# training hyperparameters
# can change learning rate or use scheduler
LEARNING_RATE = 3e-4
# finetune with less epochs to avoid forgetting
EPOCHS = 10
PATIENCE = 5
# patience- number of epochs the model continues after no improvement in validation loss

# consider other loss functions https://neptune.ai/blog/pytorch-loss-functions
criterion = nn.CrossEntropyLoss()
# don't change adam, never change dude
# optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [16]:
modellist = [mlpmodel, smallcnnmodel, resnetmodel]
# modeldict = {
#     "SmallMLP": mlpmodel,
#     "SmallCNN": smallcnnmodel,
#     "ResNet": resnetmodel
# }

modeldict = {
    "ResNet": resnetmodel
}

# modellist = [mlpmodel, smallcnnmodel]

historylist = []
stopepochs = []

for modelname, model in modeldict.items():
    print(f"Training {modelname}...")
# for _, model in enumerate(modellist):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    # try to integrate into wandb instead of storing this so we can have a pretty dashboard?
    history = {
        "train_loss": [],
        "val_loss": [],
        "train_acc": [],
        "val_acc": [],
        "val_auc": []
    }

    best_auc = -np.inf
    best_state = None
    bad_epochs = 0

    print("\nStarting training...\n")
    for epoch in tqdm(range(1, EPOCHS + 1)):
        # TODO: Train for one epoch
        tr_loss, tr_acc, _, _, _ = run_epoch(train_loader, model, criterion, optimizer=optimizer, train=True, device=device)

        # TODO: Validate
        va_loss, va_acc, va_sens, va_spec, va_auc = run_epoch(val_loader, model, criterion, optimizer=None, train=False,device=device)

        # TODO: Store metrics
        history["train_loss"].append(tr_loss)
        history["val_loss"].append(va_loss)
        history["train_acc"].append(tr_acc)
        history["val_acc"].append(va_acc)
        history["val_auc"].append(va_auc)

        print(f"Epoch {epoch:02d}: "
            f"train_loss={tr_loss:.4f} "
            f"val_loss={va_loss:.4f} "
            f"val_acc={va_acc:.3f} "
            f"val_auc={va_auc:.3f}")

        # TODO: Early stopping logic
        if va_auc > best_auc + 1e-4:
            # TODO: Update the best AUC
            best_auc = va_auc
            best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            bad_epochs = 0
        else:
            bad_epochs += 1
            if bad_epochs >= PATIENCE:
                print(f"\nEarly stopping at epoch {epoch}")
                break

    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)
        print(f"\nRestored best model (val_auc={best_auc:.4f})")

    historylist.append(history)
    stopepochs.append(epoch)

Training ResNet...

Starting training...



 10%|█         | 1/10 [01:13<10:58, 73.22s/it]

Epoch 01: train_loss=0.9269 val_loss=241.1983 val_acc=0.705 val_auc=0.743


 20%|██        | 2/10 [02:26<09:44, 73.03s/it]

Epoch 02: train_loss=0.7624 val_loss=1.8275 val_acc=0.693 val_auc=0.790


...

Epoch 08: train_loss=0.8819 val_loss=0.7600 val_acc=0.700 val_auc=0.776


 90%|█████████ | 9/10 [10:57<01:13, 73.10s/it]

Epoch 09: train_loss=0.9084 val_loss=0.8839 val_acc=0.648 val_auc=0.787


100%|██████████| 10/10 [12:10<00:00, 73.09s/it]

Epoch 10: train_loss=0.7530 val_loss=0.5842 val_acc=0.724 val_auc=0.835

Restored best model (val_auc=0.8639)


In [17]:
# finally, evaluate on test set

# for _, model in tqdm(enumerate(modellist)):
for modelname, model in modeldict.items():
    print(f"\nEvaluating {modelname}...")
    model.to(device)
    _, va_acc, va_sens, va_spec, va_auc = run_epoch(test_loader, model, criterion, train=False, device=device)
    print(f"Final Validation Performance:")
    print(f"  AUC:         {va_auc:.4f}")
    print(f"  Accuracy:    {va_acc:.4f}")
    print(f"  Sensitivity: {va_sens:.4f}")
    print(f"  Specificity: {va_spec:.4f}")


Evaluating ResNet...
Final Validation Performance:
  AUC:         0.8219
  Accuracy:    0.5002
  Sensitivity: 0.0000
  Specificity: 1.0000
